In [3]:
import numpy as np
import pandas as pd
from pandas_datareader.data import DataReader
import datetime
import os
import glob
import time

import plotly
plotly.tools.set_credentials_file(username='peryp7', api_key='tzTZVxLkNcyAEtsDCfvz')

In [6]:
states_list = pd.read_csv('states_list.csv')

state_files = [i for i in glob.glob('state_data/*.csv')]
print(state_files)

['state_data\\AK1.csv', 'state_data\\AL1.csv', 'state_data\\AL2.csv', 'state_data\\AR1.csv', 'state_data\\AR2.csv', 'state_data\\AZ1.csv', 'state_data\\AZ2.csv', 'state_data\\CA1.csv', 'state_data\\CA2.csv', 'state_data\\CO1.csv', 'state_data\\CO2.csv', 'state_data\\CT1.csv', 'state_data\\CT2.csv', 'state_data\\DC1.csv', 'state_data\\DC2.csv', 'state_data\\DE1.csv', 'state_data\\DE2.csv', 'state_data\\FL1.csv', 'state_data\\FL2.csv', 'state_data\\GA1.csv', 'state_data\\GA2.csv', 'state_data\\HI1.csv', 'state_data\\HI2.csv', 'state_data\\IA1.csv', 'state_data\\IA2.csv', 'state_data\\ID1.csv', 'state_data\\ID2.csv', 'state_data\\IL1.csv', 'state_data\\IL2.csv', 'state_data\\IN1.csv', 'state_data\\IN2.csv', 'state_data\\KS1.csv', 'state_data\\KS2.csv', 'state_data\\KY1.csv', 'state_data\\KY2.csv', 'state_data\\LA1.csv', 'state_data\\LA2.csv', 'state_data\\MA1.csv', 'state_data\\MA2.csv', 'state_data\\MD1.csv', 'state_data\\MD2.csv', 'state_data\\ME1.csv', 'state_data\\MI1.csv', 'state_dat

In [31]:
state_mean_daily_returns = list()

for s in states_list.Abbreviation:
    #print s
    state = pd.DataFrame()
    
    for f in state_files:
        if f[11:13] == s: # if state file name matches state abbreviation
            #print f
            state = pd.concat([state,pd.read_csv(f)])
            
    # filter the data only to keep publicly traded companies with market cap less than 1 billion and more than 10k
    state = state[state.MarketCap <1000000000]
    state = state[state.MarketCap >10000]
    # remove the companies with 'n/a' entry in sector - most n/a in list are not operating companies but just investment funds
    state = state[state.Sector != 'n/a']
    
    comp_mean_daily_returns = list()
    # for every remaining company
    for i in state.Symbol:
        # get financial data from Google for the entire year 2016
        comp = DataReader(i, 'google', datetime.date(2016,1,1), datetime.date(2016,12,31))
        #comp = DataReader(i, 'yahoo', datetime.date(2016,1,1), datetime.date(2016,12,31))
        print(s, i)
        if not comp.empty:
            
            # convert date index in comp dataframe to usable feature
            comp['date_usable'] = comp.index.to_series()
            
            # construct new dataframe with new features that will let us calculate close price increase per day
            new_comp = pd.DataFrame()
            
            # create 'yesterday' date and close price columns

            close_yday = np.array(comp.Close[:len(comp)-1])
            new_comp['Close_yday'] = close_yday
            date_yday = np.array(comp.date_usable[:len(comp)-1])
            new_comp['Date_yday'] = pd.to_datetime(date_yday)
            
            # create 'today' date and close price columns
            close_tday = np.array(comp.Close[1:])
            new_comp['Close_tday'] = close_tday
            date_tday = np.array(comp.date_usable[1:])
            new_comp['Date_tday'] = pd.to_datetime(date_tday)
            
            # calc how many days elapsed between yday and tday
            new_comp['delta_time'] = (np.array(new_comp.Date_tday) - np.array(new_comp.Date_yday))
            new_comp['no_days'] = np.array(new_comp['delta_time']) / np.timedelta64(1, 's') / (60*60*24)
            
            # get relative change in close price between tday and yday
            new_comp['ratio_close'] = new_comp.Close_tday/new_comp.Close_yday
            # get geometric mean of daily returns
            comp_mean_daily_returns.append(np.power(np.product(new_comp.ratio_close)[0],(1/len(new_comp.no_days)))-1)
            
#         time.sleep(5) 
    
    # append the mean for companies in that state to list
    state_mean_daily_returns.append(np.mean(comp_mean_daily_returns))

AL AUBN
AL BCRX
AL CPSI
AL FUSB
AL HIBB
AL IPCC
AL NCOM
AL NSEC
AL OTEL
AK ALSK
AK NRIM
AZ AVHI
AZ AOBC
AZ ASYS
AZ MRAM
AZ GWRS
AZ HTGM
AZ INSY
AZ SNAK
AZ KONA
AZ LLNW
AZ LIVE
AZ RIBT
AZ SNES
AZ JYNT
AZ PRSC
AZ TPIC
AZ UTI
AR CRMT
AR ARCB
AR BSF
AR PTSI
AR USAK
AR WIN
CA ACTG
CA ARAY
CA ACRX
CA AKAO
CA ADMS
CA ADMP
CA IOTS
CA ADOM
CA ADRO
CA ADVM
CA AEHR
CA AMTX
CA AEMD
CA AIRG
CA ATEC
CA AMRK
CA AMRB
CA AMPH
CA AMRS
CA ANAB
CA ANTH
CA APEN
CA APRI
CA AQMS
CA ARDM
CA RKDA
CA ARDX
CA ARNA
CA ARWR
CA ATRA
CA ATOM
CA LIFE
CA BOLD
CA ABTL
CA AVNW
CA AVGR
CA AXTI
CA RILY
CA BOCH
CA BMRC
CA BEBE
CA BGFV
CA BIOC
CA BIOL
CA BMRA
CA BJRI
CA WIFI
CA BRID
CA BVSN
CA BYFC
CA CDZI
CA PRSS
CA CAMP
CA CFNB
CA CALA
CA CAPR
CA CPST
CA CDNA
CA CATS
CA CBMG
CA CVCY
CA CERS
CA KOOL
CA CEVA
CA CCXI
CA CHKE
CA CDXC
CA CDTX
CA CDTI
CA CLNE
CA CDXS
CA CHRS
CA COHU
CA CLCT
CA CBMX
CA CWBC
CA CNAT
CA CPSS
CA CORI
CA CRVL
CA CRVS
CA CYRX
CA CUNB
CA CUTR
CA CBAY
CA CYRN
CA CYTK
CA CTMX
CA CYTX
CA CYTR
CA DJCO
CA 

NJ PDVW
NJ PGC
NJ PPSI
NJ COOL
NJ PTCT
NJ RLOG
NJ RCMT
NJ ROKA
NJ SCYX
NJ SSNT
NJ SITO
NJ SNGX
NJ SSFN
NJ SNBC
NJ SBBX
NJ SNCR
NJ TRHC
NJ TLGT
NJ TRCB
NJ UNTY
NJ VLRX
NJ VLGEA
NJ WSTG
NJ GCAP
NJ GNE
NJ HIL
NJ HOV
NJ IDT
NJ MG
NJ MOV
NJ NRT
NJ PHH
NJ RTEC
NJ SLD
NJ UMH
NJ VSI
NM BCTF
NM EMKR
NM SGLB
NY TURN
NY FLWS
NY ACET
NY AGFS
NY AKTX
NY ALJJ
NY AMMA
NY AMBC
NY AVXL
NY ANDA
NY ANGO
NY APDN
NY ARKR
NY ARTX
NY AROW
NY ATRO
NY ATNX
NY AAPC
NY ATXI
NY AHPA
NY AXAR
NY AXSM
NY AZRX
NY BHAC
NY BYSI
NY BSTC
NY BLVD
NY BCLI
NY BDGE
NY CLBS
NY ABCD
NY TAST
NY CARV
NY CETX
NY CKPT
NY CEMI
NY CHMG
NY CBLI
NY JVA
NY CMCO
NY CHUBK
NY CTG
NY CMTL
NY CFRX
NY COWN
NY CVV
NY DCOM
NY EGLE
NY EACQ
NY EEI
NY ELEC
NY ELECU
NY ESBK
NY NYNY
NY ESQ
NY FALC
NY FRGI
NY FISI
NY FNJN
NY FFIC
NY FONR
NY FH
NY FMCI
NY FEIM
NY FSBC
NY FSNN
NY ROCK
NY GLBR
NY GPAC
NY SELF
NY GPIA
NY GCBC
NY GRIF
NY GYRO
NY HNH
NY HDNG
NY HMNY
NY HSON
NY HDSN
NY ICON
NY IFMK
NY INDU
NY INSE
NY ITCI
NY IRDM
NY IRDMB
NY MAYS
NY KERX
N

RemoteDataError: Unable to read URL: http://www.google.com/finance/historical?q=SPOK&startdate=Jan+01%2C+2016&enddate=Dec+31%2C+2016&output=csv

In [60]:
# save the list to dataframe and into csv file
#final_data = states_list
# final_data
#final_data['state_mean_daily_returns'] = state_mean_daily_returns
#final_data.to_csv('final_data.csv')

In [61]:
df = pd.read_csv('final_data.csv')

In [62]:
# create a plot

for col in df.columns:
    df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

# df['text'] = df['state'] + '<br>' +\
#     'Beef '+df['beef']+' Dairy '+df['dairy']+'<br>'+\
#     'Fruits '+df['total fruits']+' Veggies ' + df['total veggies']+'<br>'+\
#     'Wheat '+df['wheat']+' Corn '+df['corn']
    
df['text'] = df['Abbreviation'] + '<br>' +\
    'Mean daily return in '+df['state_mean_daily_returns']
    
data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['Abbreviation'],
        z = df['state_mean_daily_returns'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "fractional increase"
        )
    ) ]

layout = dict(
        title = 'Mean daily return for mid-size companies by state<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)',
        ),
    )

fig = dict( data=data, layout=layout )

url = plotly.plotly.plot( fig, filename='d3-cloropleth-map' )

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~peryp7/0 or inside your plot.ly account where it is named 'd3-cloropleth-map'
